# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project, we’ll be using foursquare API to get the data regarding two cities, New York City an The city of Toronto. Then we’ll find their neighborhoods and cluster them according to their venue categories. As a result, we can find out which portion of each city is similar to the other and which are not. 

If anyone wants to examine a portion of a particular city, say, someone wants to check out a restaurant in a particular city then what is the best possibility to find that kind of restaurant to the other city and in which portion. Neighborhoods that are in the same cluster will be similar to their categories. 

## Data <a name="data"></a>

Firstly we’ll download the data with the borough, neighborhood, latitude, and longitude of each city. Then
we’ll **merge** those two datasets into one and by using **foursquare API** we’ll **collect** the **venues** for each **neighborhood**. Each venue consists of **four** items. **Unique id, name, categories, and location.** Our main focus will be on venue categories. We’ll **extract** different **venue categories** along with the venues and
using them we’ll **cluster** the full dataset.

In [1]:
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

## Preparing the data

In [2]:
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
#newyork_data

In [4]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_newyork = pd.DataFrame(columns=column_names)
neighborhoods_newyork

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods_newyork = neighborhoods_newyork.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
print(neighborhoods_newyork.shape)
neighborhoods_newyork.head()

(306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
toronto_data = pd.read_csv('toronto.csv')
toronto_data['Neighborhood']=toronto_data['Neighborhood'].str.replace('/',',')
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [11]:
toronto_data.shape

(103, 3)

#### Finding the geographical coordinates of the different region Toronto according to the postal code

In [12]:
import geocoder

def lat_long(postal):
    
    lat_lng_coords = None
    postal_code = postal
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [13]:
toronto_data['Latitude'], toronto_data['Longitude'] = zip(*toronto_data['PostalCode'].apply(lat_long))

toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390


In [14]:
toronto_data = toronto_data.drop(columns = 'PostalCode', axis = 1)
print(toronto_data.shape)
toronto_data.head()

(103, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.752935,-79.335641
1,North York,Victoria Village,43.728102,-79.311890
2,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390


#### Merging Newyork and Toronto dataframe into one

In [15]:
merged_data = pd.concat([neighborhoods_newyork, toronto_data], ignore_index = True)
#m_data = pd.DataFrame(merged_data, reset_index=True)
#merged_data = merged_data.reindex_like(neighborhoods_newyork)
print(merged_data.shape)
#merged_data

(409, 4)


## Methodology <a name="methodology"></a>

> - We'll collect the latitude and longitude for the Newyork city and The city of Toronto.
> - Then we will use the **foursquare API** and explore the neighborhoods of these cities within 500 radius.
> - From API call we'll collect the neighborhoods name along with their locations and and venue categories.
> - Find out the unique categories and create a onehot dataframe with the columns of the names of venue categories.
> - Then keep the top veneues according to venues and specify the num of cluster(cluster = 10).
> - Apply **K-means** clustering algorithm to find out the similar neighborhoods
> - Finding the number of venues in a cluster and information regarding their name and show them in the folium map

## Analysis

In [16]:
address_newyork = 'New York City, NY'

geolocator = Nominatim(user_agent="Coursera-Capstone-Project")
location = geolocator.geocode(address_newyork)
latitude_newyork = location.latitude
longitude_newyork = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_newyork, longitude_newyork))



address_toronto = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Coursera-Capstone-Project")
location = geolocator.geocode(address_toronto)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Visualizing the map of the Newyork with its neighborhoods

In [17]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude_newyork, longitude_newyork], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_newyork['Latitude'], neighborhoods_newyork['Longitude'], neighborhoods_newyork['Borough'], neighborhoods_newyork['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Map of the city of Toronto along with it's neighborhoods

In [18]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Visualizing the map by merging two cities

In [19]:
latitude_center = (latitude_newyork+latitude_toronto)/2
longitude_center = (longitude_newyork+longitude_toronto)/2
map_center = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 


for lat, lng, label in zip(neighborhoods_newyork['Latitude'], neighborhoods_newyork['Longitude'], neighborhoods_newyork['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  
    

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_center)  

#displaying map
map_center


In [20]:
CLIENT_ID = 'X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU'
CLIENT_SECRET = '4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB'
VERSION = '20200420' # Foursquare API version
LIMIT = 100
radius = 500

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print(LIMIT)
print(radius)

CLIENT_ID: X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU
CLIENT_SECRET:4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB
100
500


In [21]:
neighborhood_latitude = merged_data.loc[0,'Latitude'] 
neighborhood_longitude = merged_data.loc[0,'Longitude']
neighborhood_name = merged_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url


Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


'https://api.foursquare.com/v2/venues/explore?&client_id=X5OBM0WZCSVVJASBUED5CUOTIGF3D4UKUTXLQ3G44JTG3RLU&client_secret=4OMKQ2OOFVOV1BC1ZMQAHN0QZAPUICT3SOUAI2TKDMF4ZEBB&v=20200420&ll=40.89470517661,-73.84720052054902&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
#results

#### Getting the venues regarding to the location of each neighborhood

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues, venues_list)

In [24]:
(merged_venues, venues_list) = getNearbyVenues(names=merged_data['Neighborhood'],
                                   latitudes=merged_data['Latitude'],
                                   longitudes=merged_data['Longitude']
                                  )

In [25]:
merged_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [26]:
merged_venues.shape  # Total number of venues

(12030, 7)

In [27]:
merged_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",10,10,10,10,10,10
Allerton,27,27,27,27,27,27
Annadale,9,9,9,9,9,9
Arden Heights,5,5,5,5,5,5
Arlington,6,6,6,6,6,6
Arrochar,21,21,21,21,21,21
Arverne,17,17,17,17,17,17
Astoria,97,97,97,97,97,97


#### Finding unique categories of the venues

In [28]:
print('There are {} uniques categories.'.format(len(merged_venues['Venue Category'].unique())))

There are 457 uniques categories.


In [29]:
# one hot encoding
merged_onehot = pd.get_dummies(merged_venues[['Venue Category']], prefix="", prefix_sep="")
merged_onehot['Neighborhood'] = merged_venues['Neighborhood']
print(merged_onehot.shape)
merged_onehot.head()

(12030, 457)


,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Go

In [30]:
loc = merged_onehot.columns.get_loc('Neighborhood')
fixed_columns = [merged_onehot.columns[loc]] + list(merged_onehot.columns[:loc]) + list(merged_onehot.columns[loc+1:])
merged_onehot = merged_onehot[fixed_columns]

print(merged_onehot.shape)
merged_onehot.head()

(12030, 457)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Resta

In [32]:
merged_grouped = merged_onehot.groupby('Neighborhood').mean().reset_index()
merged_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Resta

In [33]:
merged_grouped.shape

(396, 457)

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
merged_venues_sorted = pd.DataFrame(columns=columns)
merged_venues_sorted['Neighborhood'] = merged_grouped['Neighborhood']

for ind in np.arange(merged_grouped.shape[0]):
    merged_venues_sorted.iloc[ind, 1:] = return_most_common_venues(merged_grouped.iloc[ind, :], num_top_venues)

merged_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Badminton Court,Supermarket,Sushi Restaurant,Skating Rink,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
1,"Alderwood , Long Branch",Pub,Dance Studio,Sandwich Place,Pharmacy,Coffee Shop,Athletics & Sports,Gym,Pizza Place,Gas Station,Convenience Store
2,Allerton,Pizza Place,Cosmetics Shop,Supermarket,Bus Station,Deli / Bodega,Department Store,Breakfast Spot,Pharmacy,Spanish Restaurant,Fried Chicken Joint
3,Annadale,Pub,Pizza Place,Diner,Train Station,Pharmacy,Restaurant,Dance Studio,Sports Bar,Food,Ethiopian Restaurant
4,Arden Heights,Deli / Bodega,Pharmacy,Coffee Shop,Pizza Place,Bus Stop,Yoga Studio,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service


In [37]:
kclusters = 10

merged_grouped_clustering = merged_grouped.drop('Neighborhood', 1)

merged_grouped_clustering.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Go

#### Applying K-means clustering algorithm

In [38]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_
labels[0:5]

array([0, 9, 9, 9, 8], dtype=int32)

## Result & Discussion <a name="results"></a>

#### Assigning lables to the each neighborhood

In [39]:
merged_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
fully_merged = merged_data

# merge  merged_data with merged_venues_sorted to add latitude/longitude for each neighborhood
fully_merged = fully_merged.join(merged_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(fully_merged.shape)
fully_merged.head()

(409, 15)


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0.0,Pharmacy,Donut Shop,Laundromat,Gas Station,Ice Cream Shop,Dessert Shop,Sandwich Place,Factory,Exhibit,Yoga Studio
1,Bronx,Co-op City,40.874294,-73.829939,9.0,Bus Station,Restaurant,Bagel Shop,Pizza Place,Park,Mattress Store,Salon / Barbershop,Pharmacy,Baseball Field,Discount Store
2,Bronx,Eastchester,40.887556,-73.827806,0.0,Caribbean Restaurant,Deli / Bodega,Diner,Cosmetics Shop,Juice Bar,Bowling Alley,Bus Station,Bus Stop,Fast Food Restaurant,Metro Station
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Plaza,River,Bus Station,Yoga Studio,Field,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space
4,Bronx,Riverdale,40.890834,-73.912585,6.0,Park,Bus Station,Food Truck,Baseball Field,Gym,Plaza,Playground,Home Service,Bank,English Restaurant


In [40]:

#fully_merged.isna()

fully_merged.dropna(subset = ['Cluster Labels'], inplace = True)
#fully_merged['Cluster Labels'] = fully_merged['Cluster Labels'].astype(int)

In [41]:
fully_merged.shape

(406, 15)

In [42]:
fully_merged['Cluster Labels'] = fully_merged['Cluster Labels'].astype(int)

#### Final map after clustering

In [43]:
# create map
map_clusters = folium.Map(location=[latitude_center, longitude_center], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
#print(x)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
#print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#print(colors)
rainbow = [colors.rgb2hex(i) for i in colors_array]
#print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fully_merged['Latitude'], fully_merged['Longitude'], fully_merged['Neighborhood'], fully_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Number of neighborhoods in a signle cluster and it's corresponding folium map

In [45]:
cluster1 = fully_merged.loc[fully_merged['Cluster Labels'] == 0, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 1".format(cluster1.shape[0]))
#cluster1

There are 164 neighborhoods in cluster 1


In [46]:
map_cluster1 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster1['Latitude'], cluster1['Longitude'], cluster1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster1) 
    
map_cluster1

In [47]:
cluster2 = fully_merged.loc[fully_merged['Cluster Labels'] == 1, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 2".format(cluster2.shape[0]))
#cluster2

There are 4 neighborhoods in cluster 2


In [48]:
map_cluster2 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster2['Latitude'], cluster2['Longitude'], cluster2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#4856fb',
        fill=True,
        fill_color='#4856fb',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster2) 
    
map_cluster2

In [49]:
cluster3 = fully_merged.loc[fully_merged['Cluster Labels'] == 2, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 3".format(cluster3.shape[0]))
#cluster3

There are 4 neighborhoods in cluster 3


In [50]:
map_cluster3 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster3['Latitude'], cluster3['Longitude'], cluster3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#10a2f0',
        fill=True,
        fill_color='#10a2f0',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster3) 
    
map_cluster3

In [51]:
cluster4 = fully_merged.loc[fully_merged['Cluster Labels'] == 3, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 4".format(cluster4.shape[0]))
#cluster4

There are 1 neighborhoods in cluster 4


In [52]:
map_cluster4 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster4['Latitude'], cluster4['Longitude'], cluster4['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#2adddd',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster4) 
    
map_cluster4

In [53]:
cluster5 = fully_merged.loc[fully_merged['Cluster Labels'] == 4, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 5".format(cluster5.shape[0]))
#cluster5

There are 126 neighborhoods in cluster 5


In [54]:
map_cluster5 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster5['Latitude'], cluster5['Longitude'], cluster5['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#62fbc4',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster5) 
    
map_cluster5

In [55]:
cluster6 = fully_merged.loc[fully_merged['Cluster Labels'] == 5, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 6".format(cluster6.shape[0]))
#cluster6

There are 1 neighborhoods in cluster 6


In [56]:
map_cluster6 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster6['Latitude'], cluster6['Longitude'], cluster6['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#9cfba4',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster6) 
    
map_cluster6

In [57]:
cluster7 = fully_merged.loc[fully_merged['Cluster Labels'] == 6, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 7".format(cluster7.shape[0]))
#cluster7

There are 15 neighborhoods in cluster 7


In [58]:
map_cluster7 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster7['Latitude'], cluster7['Longitude'], cluster7['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#d4dd80',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster7) 
    
map_cluster7

In [59]:
cluster8 = fully_merged.loc[fully_merged['Cluster Labels'] == 7, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 8".format(cluster8.shape[0]))
#cluster8

There are 1 neighborhoods in cluster 8


In [60]:
map_cluster8 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster8['Latitude'], cluster8['Longitude'], cluster8['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#ffa256',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster8) 
    
map_cluster8

In [61]:
cluster9 = fully_merged.loc[fully_merged['Cluster Labels'] == 8, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 9".format(cluster9.shape[0]))
#cluster9

There are 18 neighborhoods in cluster 9


In [62]:
map_cluster9 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster9['Latitude'], cluster9['Longitude'], cluster9['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#ff562c',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster9) 
    
map_cluster9

In [63]:
cluster10 = fully_merged.loc[fully_merged['Cluster Labels'] == 9, fully_merged.columns[[1,2,3] + list(range(5, fully_merged.shape[1]))]]
print("There are {} neighborhoods in cluster 10".format(cluster10.shape[0]))
#cluster10

There are 72 neighborhoods in cluster 10


In [64]:
map_cluster10 = folium.Map(location=[latitude_center, longitude_center], zoom_start=7) 

for lat, lng, label in zip(cluster10['Latitude'], cluster10['Longitude'], cluster10['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='#ff0000',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster10) 
    
map_cluster10

> Total number of 10 clusters are there, and each cluster consists some values. In cluster number 1,2,3,5,7,9 & 10 consist of values which are on both cities which in case we can say are similar places. but cluster number 4,6 & 8 consist of only one values. So, they are the unique values with unique characteristics which are not similar to the others

## Conclusion <a name = "conclusion"/>

> In this project we wanted to cluster those places in both the Newyork city and the city of Toronto which are more similar to each other. So that, if we are in a part of a city we can assume that in what part of the other city can be the similar like present city. We have used total 10 clusters to apply the algorithm and got the result. Apart from that we could use less number of clusters to get more compact values as some of our clusters have only one item in it.